In [2]:
import numpy as np
import pandas as pd
from shapely import wkt
import geopandas as gpd
from pathlib import Path
import matplotlib.pyplot as plt
from shapely import wkt
from shapely.geometry import Polygon
import json
from tqdm import tqdm
import os

In [1]:
RAW_DIR = "../../../data/tm/1-raw/"
PROCESSED_DIR = "../../../data/tm/2-processed/"

# Crop buildings from country file

This notebook does the following steps: 

(1) crops the single country-wide buildings `geojsonl` (link to file [here](https://github.com/microsoft/IdMyPhBuildingFootprints)) to small batches of geojson files

(2) extracts intersecting buildings within project-included city bounds, and finally 

(3) combines all city building extracts to a single file.

## 1. Create small batches

In [3]:
filepath = f"{RAW_DIR}ms-open-buildings/philippines.geojsonl"
total_lines = sum(1 for line in open(filepath))
batch_size = 500000
nbatches = int(total_lines / batch_size + 1)
print(f"Total lines: {total_lines}, Number of batches: {nbatches}")

Total lines: 17421764, Number of batches: 35


In [6]:
with open(filepath, "r") as f:
    counter = 0
    batch = []
    for line in tqdm(f, total=total_lines):
        feature = json.loads(line)
        batch.append(feature)
        counter += 1
        if counter % batch_size == 0:
            batch_id = int(counter / batch_size)
            print(f"Saving batch {batch_id}...", end="")
            gdf = gpd.GeoDataFrame.from_features(batch)
            output_file = (
                f"{PROCESSED_DIR}ms-open-buildings/batch/batch_{batch_id}.geojson"
            )
            gdf.to_file(output_file, driver="GeoJSON")
            # clear batch content
            batch = []
            print("DONE!")
    # for last batch
    if batch:
        gdf = gpd.GeoDataFrame.from_features(batch)
        output_file = f"{PROCESSED_DIR}ms-open-buildings/batch/batch_{int((counter-1)/batch_size)+1}.geojson"
        gdf.to_file(output_file, driver="GeoJSON")

  3%|▎         | 498656/17421764 [00:07<02:33, 109933.26it/s]

Saving batch 1...


  6%|▌         | 998560/17421764 [01:21<02:30, 108978.98it/s]

Saving batch 2...


  9%|▊         | 1490024/17421764 [02:33<03:05, 86035.90it/s] 

...

 95%|█████████▍| 16489727/17421764 [40:01<00:07, 117013.67it/s]

Saving batch 33...


 98%|█████████▊| 16995633/17421764 [41:01<00:06, 63538.26it/s] 

Saving batch 34...


100%|██████████| 17421764/17421764 [41:57<00:00, 6921.58it/s]  


## 2. Check which batch files are within included cities

In [4]:
# Read cities bounds
bounds_gdf = gpd.read_file(
    "../../../data/1-admin-bounds/target_admin_bounds.shp"
).set_crs(4326)

In [5]:
# Loop over all batch, append and save part of batches within a city
ms_gdf_list = []
for batch in tqdm(np.arange(1, nbatches + 1)):
    ms_gdf = gpd.read_file(
        f"{PROCESSED_DIR}ms-open-buildings/batch/batch_{batch}.geojson"
    )
    ms_gdf = ms_gdf.set_geometry("geometry")
    ms_gdf = ms_gdf.set_crs(4326)

    bldgs_gdf = gpd.sjoin(ms_gdf, bounds_gdf, predicate="intersects")
    bldgs_gdf = bldgs_gdf[["ADM3_EN", "ADM4_PCODE", "geometry"]]
    print(f"Batch {batch} building count: {len(bldgs_gdf)}")

    if len(bldgs_gdf) > 0:
        ms_gdf_list.append(bldgs_gdf)

  3%|▎         | 1/35 [00:49<28:02, 49.50s/it]

Batch 1 building count: 4630


  6%|▌         | 2/35 [01:41<28:11, 51.25s/it]

Batch 2 building count: 174282


  9%|▊         | 3/35 [02:31<26:54, 50.47s/it]

...

Batch 33 building count: 7910


 97%|█████████▋| 34/35 [25:32<00:43, 43.38s/it]

Batch 34 building count: 0


100%|██████████| 35/35 [26:08<00:00, 44.81s/it]

Batch 35 building count: 0


In [6]:
all_ms_gdf = pd.concat(ms_gdf_list)
all_ms_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 794817 entries, 437436 to 372916
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   ADM3_EN     794817 non-null  object  
 1   ADM4_PCODE  794817 non-null  object  
 2   geometry    794817 non-null  geometry
dtypes: geometry(1), object(2)
memory usage: 24.3+ MB


## 3. Read filtered files and save by city

In [7]:
city_names = all_ms_gdf["ADM3_EN"].unique()
short_city_names = [
    "".join([s.lower() for s in c.split(" ") if len(s) > 4]) for c in city_names
]
short_city_names = ["cdo" if s == "cagayan" else s for s in short_city_names]
city_names, short_city_names

(array(['Zamboanga City', 'Davao City', 'City of Navotas',
        'Cagayan de Oro City', 'Legazpi City', 'Tacloban City',
        'Dagupan City', 'City of Mandaluyong', 'City of Muntinlupa',
        'Palayan City', 'Mandaue City', 'Iloilo City'], dtype=object),
 ['zamboanga',
  'davao',
  'navotas',
  'cdo',
  'legazpi',
  'tacloban',
  'dagupan',
  'mandaluyong',
  'muntinlupa',
  'palayan',
  'mandaue',
  'iloilo'])

In [10]:
for short_city_name, city_name in tqdm(zip(short_city_names, city_names)):
    save_gdf = all_ms_gdf[all_ms_gdf["ADM3_EN"] == city_name]
    print(f"Number of bldgs for {short_city_name}: {len(save_gdf)}")
    save_gdf.to_file(
        f"{PROCESSED_DIR}ms-open-buildings/within_city/ms_lacuna_cities_bldgs_{short_city_name}_v1.geojson",
        driver="GeoJSON",
        index=False,
    )

0it [00:00, ?it/s]

Number of bldgs for zamboanga: 146792


1it [00:17, 17.46s/it]

Number of bldgs for davao: 290748


2it [00:52, 27.53s/it]

...

10it [01:25,  3.95s/it]

Number of bldgs for mandaue: 33685


11it [01:29,  3.97s/it]

Number of bldgs for iloilo: 54830


12it [01:35,  7.97s/it]
